In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score


data = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [2]:
remove_columns = ['Unnamed: 32', 'id']
data.drop(columns=remove_columns, inplace=True) 

In [3]:
x = data.drop(columns='diagnosis')
y = data['diagnosis']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [4]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [5]:
from layer.connect_layers import *
from losses.loss import *
from multi_activation.activation import *
from model.main_model import *

In [ ]:
model = Model()
model.add(DenseLayer(x_train.shape[1], 64)) 
model.add(Activation_ReLU())
model.add(DenseLayer(64, 1))              
model.add(Activation_Sigmoid())

loss_function = Loss_BinaryCrossentropy()
optimizer = Optimizer_SGD(learning_rate=0.01)

model.loss = loss_function

epochs = 1000
for epoch in range(epochs):

    y_pred = model.forward(x_train)

    loss = loss_function.calculate(y_pred, y_train)

    # Single Backpropagation, rest do the backpropagation automatically
    model.backward(y_pred, y_train)

    # Use the optimizer to update the weights and biases of each dense layer
    for layer in model.layers:
        if hasattr(layer, 'weights'): # Check if the layer has parameters to update
            optimizer.update_params(layer)

    if epoch % 100 == 0:
        accuracy = accuracy_score(y_train, (y_pred > 0.5).astype(int))

        print(f"Epoch: {epoch}, Accuracy: {accuracy:.5f}, Loss: {loss:.3f}")

Epoch: 0, Accuracy: 0.69231, Loss: 0.693
Epoch: 100, Accuracy: 0.62857, Loss: 0.679
Epoch: 200, Accuracy: 0.63956, Loss: 0.672
Epoch: 300, Accuracy: 0.84835, Loss: 0.679
Epoch: 400, Accuracy: 0.93626, Loss: 0.753
Epoch: 500, Accuracy: 0.94066, Loss: 0.918
Epoch: 600, Accuracy: 0.94505, Loss: 1.108
Epoch: 700, Accuracy: 0.95824, Loss: 1.285
Epoch: 800, Accuracy: 0.96264, Loss: 1.441
Epoch: 900, Accuracy: 0.96703, Loss: 1.578


In [7]:
y_test_pred = model.forward(x_test)
test_loss = loss_function.calculate(y_test_pred, y_test)
test_accuracy = accuracy_score(y_test, (y_test_pred > 0.5).astype(int))
print(f"Test Accuracy: {test_accuracy:.5f}, Test Loss: {test_loss:.3f}")

Test Accuracy: 0.98246, Test Loss: 1.754


### compare this with the pytorch version

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score

In [ ]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # Reshape to (n_samples, 1)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Reshape to (n_samples, 1)

In [ ]:
class BinaryClassificationModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BinaryClassificationModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

In [ ]:

model = BinaryClassificationModel(x_train_tensor.shape[1], 64, 1)

criterion = nn.BCELoss() 
optimizer = optim.SGD(model.parameters(), lr=0.01)

epochs = 1000
for epoch in range(epochs):
    y_pred = model(x_train_tensor)

    loss = criterion(y_pred, y_train_tensor)

    optimizer.zero_grad()
    loss.backward()       
    optimizer.step()      

    if epoch % 100 == 0:
        y_pred_labels = (y_pred > 0.5).float()
        accuracy = accuracy_score(y_train, y_pred_labels.detach().numpy())
        print(f"Epoch: {epoch}, Accuracy: {accuracy:.5f}, Loss: {loss.item():.3f}")

Epoch: 0, Accuracy: 0.35604, Loss: 0.738
Epoch: 100, Accuracy: 0.93846, Loss: 0.401
Epoch: 200, Accuracy: 0.94286, Loss: 0.265
Epoch: 300, Accuracy: 0.94725, Loss: 0.201
Epoch: 400, Accuracy: 0.95824, Loss: 0.166
Epoch: 500, Accuracy: 0.96044, Loss: 0.144
Epoch: 600, Accuracy: 0.96264, Loss: 0.129
Epoch: 700, Accuracy: 0.96923, Loss: 0.117
Epoch: 800, Accuracy: 0.97363, Loss: 0.108
Epoch: 900, Accuracy: 0.97802, Loss: 0.100
